In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [9]:
# health
obj = Liputan6()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 23)
init_page = 1
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[4:]

In [10]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-01-23
showbiz
page  1
https://www.liputan6.com/showbiz/indeks/2018/01/23?page=1
Insert berita  Unggul Tipis, Oji Rebut Tiket Duta Kepulauan Riau ke Top 34 LIDA
salah2
Insert berita  6 Fakta yang Selalu Dilakukan Tom Cruise di Semua Filmnya
salah2
Insert berita  Ini Permintaan Terakhir Sys NS pada Anak
salah2
Insert berita  Feby Febiola Umumkan Ultah Pernikahan, Warganet Bereaksi
salah2
Insert berita  Song Hye Kyo yang Tak Pernah Menua
salah2
Insert berita  Avengers 4 Pertemukan Samuel L Jackson dengan Aktris Ini
salah2
Insert berita  Detik-Detik Sys NS Meninggal Dunia
salah2
Insert berita  Videoklip Terakhir Mendiang Jonghyun SHINee Dirilis
salah2
Insert berita  Ini Sosok di Balik Viralnya Gaya Rambut Pasha
salah2
Insert berita  Orang Ketiga: Akankah Hubungan Aris dan Yuni Terbongkar?
salah2
Insert berita  Main ke Kuburan, Roro Fitria Ngaku Didatangi Hantu Cantik
salah2
Insert berita  Kalimat Tahlil Iringi Pemakaman Sys NS
salah2
Insert berita  Anak Langit: Usaha Milka Mencari Hir

AttributeError: 'NoneType' object has no attribute 'findAll'